In [42]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from pathlib import Path

BASE_DIR = Path.cwd()
DATASET_PATH = BASE_DIR / "dataset_filtered"
LABEL_MAP = BASE_DIR / "label_map.txt"
MODEL_NAME = BASE_DIR / "crop_disease_model.h5"
EPOCHS = 12

In [43]:
if not DATASET_PATH.exists():
    raise FileNotFoundError(
        f"Dataset folder not found at {DATASET_PATH}\n"
        "Please download the dataset and place it in the project root as 'dataset_filtered/'"
    )

In [44]:
train_datagen = ImageDataGenerator(
    preprocessing_function = tf.keras.applications.mobilenet_v2.preprocess_input,
    validation_split=0.2,
    rotation_range=25,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    fill_mode="nearest"
)

In [45]:
val_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    validation_split=0.2
)

In [46]:
train_data = train_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical",
    subset="training"
)

Found 16254 images belonging to 13 classes.


In [47]:
val_data = val_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical",
    subset="validation"
)

Found 4058 images belonging to 13 classes.


In [48]:
NUM_CLASSES = train_data.num_classes
labels = train_data.class_indices
with open(LABEL_MAP, "w") as f:
    for label, idx in labels.items():
        f.write(f"{idx}:{label}\n")

print("✅ Label map saved to", LABEL_MAP)

✅ Label map saved to d:\Lohith\BTECH CODES\Machine_Learning\Project\Crop-Disease\label_map.txt


In [49]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights="imagenet"
)
base_model.trainable = False 

In [50]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(256, activation="relu"),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(NUM_CLASSES, activation="softmax")
])

In [51]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [53]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 13)             │         3,341 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,589,261 (9.88 MB)

 Trainable params: 331,277 (1.26 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [55]:
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(train_data.classes),
    y=train_data.classes
)

class_weights = dict(enumerate(class_weights))
print("Class Weights:", class_weights)

Class Weights: {0: np.float64(1.5628846153846154), 1: np.float64(1.5628846153846154), 2: np.float64(10.2484237074401), 3: np.float64(0.7346108650456477), 4: np.float64(1.5628846153846154), 5: np.float64(0.8182641965364479), 6: np.float64(1.6408237431859478), 7: np.float64(0.8823625210357744), 8: np.float64(0.9323696437790397), 9: np.float64(1.1123733917328222), 10: np.float64(0.29171901360422126), 11: np.float64(4.181631077952148), 12: np.float64(0.9821741494954378)}


In [56]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=4, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint(MODEL_NAME, save_best_only=True),
    tf.keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.3)
]

In [57]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=EPOCHS,
    class_weight=class_weights,
    callbacks=callbacks
)

Epoch 1/12
508/508 ━━━━━━━━━━━━━━━━━━━━ 0s 587ms/step - accuracy: 0.5285 - loss: 1.5525

508/508 ━━━━━━━━━━━━━━━━━━━━ 361s 707ms/step - accuracy: 0.6670 - loss: 1.0932 - val_accuracy: 0.8346 - val_loss: 0.5313 - learning_rate: 3.0000e-04
Epoch 2/12
508/508 ━━━━━━━━━━━━━━━━━━━━ 0s 398ms/step - accuracy: 0.8056 - loss: 0.6129

508/508 ━━━━━━━━━━━━━━━━━━━━ 235s 462ms/step - accuracy: 0.8157 - loss: 0.5728 - val_accuracy: 0.8561 - val_loss: 0.4316 - learning_rate: 3.0000e-04
Epoch 3/12
508/508 ━━━━━━━━━━━━━━━━━━━━ 0s 396ms/step - accuracy: 0.8445 - loss: 0.4862

508/508 ━━━━━━━━━━━━━━━━━━━━ 234s 460ms/step - accuracy: 0.8511 - loss: 0.4631 - val_accuracy: 0.8810 - val_loss: 0.3538 - learning_rate: 3.0000e-04
Epoch 4/12
508/508 ━━━━━━━━━━━━━━━━━━━━ 0s 396ms/step - accuracy: 0.8632 - loss: 0.4205

508/508 ━━━━━━━━━━━━━━━━━━━━ 232s 457ms/step - accuracy: 0.8666 - loss: 0.4131 - val_accuracy: 0.8898 - val_loss: 0.3281 - learning_rate: 3.0000e-04
Epoch 5/12
508/508 ━━━━━━━━━━━━━━━━━━━━ 226s 445ms/step - accuracy: 0.8753 - loss: 0.3863 - val_accuracy: 0.8701 - val_loss: 0.3656 - learning_rate: 3.0000e-04
Epoch 6/12
508/508 ━━━━━━━━━━━━━━━━━━━━ 0s 401ms/step - accuracy: 0.8851 - loss: 0.3516

508/508 ━━━━━━━━━━━━━━━━━━━━ 236s 464ms/step - accuracy: 0.8851 - loss: 0.3550 - val_accuracy: 0.8970 - val_loss: 0.3063 - learning_rate: 3.0000e-04
Epoch 7/12
508/508 ━━━━━━━━━━━━━━━━━━━━ 226s 446ms/step - accuracy: 0.8912 - loss: 0.3300 - val_accuracy: 0.8812 - val_loss: 0.3328 - learning_rate: 3.0000e-04
Epoch 8/12
508/508 ━━━━━━━━━━━━━━━━━━━━ 0s 365ms/step - accuracy: 0.8873 - loss: 0.3302

508/508 ━━━━━━━━━━━━━━━━━━━━ 216s 425ms/step - accuracy: 0.8919 - loss: 0.3168 - val_accuracy: 0.8938 - val_loss: 0.2980 - learning_rate: 3.0000e-04
Epoch 9/12
508/508 ━━━━━━━━━━━━━━━━━━━━ 0s 379ms/step - accuracy: 0.8993 - loss: 0.3107

508/508 ━━━━━━━━━━━━━━━━━━━━ 224s 441ms/step - accuracy: 0.9022 - loss: 0.2982 - val_accuracy: 0.9000 - val_loss: 0.2798 - learning_rate: 3.0000e-04
Epoch 10/12
508/508 ━━━━━━━━━━━━━━━━━━━━ 224s 442ms/step - accuracy: 0.9041 - loss: 0.2835 - val_accuracy: 0.9012 - val_loss: 0.2815 - learning_rate: 3.0000e-04
Epoch 11/12
508/508 ━━━━━━━━━━━━━━━━━━━━ 222s 436ms/step - accuracy: 0.9078 - loss: 0.2714 - val_accuracy: 0.9022 - val_loss: 0.2807 - learning_rate: 3.0000e-04
Epoch 12/12
508/508 ━━━━━━━━━━━━━━━━━━━━ 0s 373ms/step - accuracy: 0.9192 - loss: 0.2402

508/508 ━━━━━━━━━━━━━━━━━━━━ 219s 430ms/step - accuracy: 0.9213 - loss: 0.2371 - val_accuracy: 0.9096 - val_loss: 0.2629 - learning_rate: 9.0000e-05


In [58]:
model.save("crop_disease_model.keras")